```
Copyright 2021 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Logistic Regression on Allstate Dataset

## Background 

The goal of this competition is to predict Bodily Injury Liability Insurance claim payments based on the characteristics of the insured’s vehicle. 

## Source

The raw dataset can be obtained directly from the [Allstate Claim Prediction Challenge](https://www.kaggle.com/c/ClaimPredictionChallenge).

In this example, we download the dataset directly from Kaggle using their API. In order for to work work, you must:
1. Login into Kaggle and accept the [competition rules](https://www.kaggle.com/c/ClaimPredictionChallenge/rules).
2. Folow [these instructions](https://www.kaggle.com/docs/api) to install your API token on your machine.

## Goal
The goal of this notebook is to illustrate how Snap ML can accelerate training of a logistic regression model on this dataset.

## Code

In [1]:
cd ../../

/Users/tpa/Code/snapml-examples/examples


In [2]:
CACHE_DIR='cache-dir'

In [3]:
import numpy as np
import time
from datasets import Allstate
from sklearn.linear_model import LogisticRegression
from snapml import LogisticRegression as SnapLogisticRegression
from sklearn.metrics import roc_auc_score

In [4]:
X_train, X_test, y_train, y_test = Allstate(cache_dir=CACHE_DIR).get_train_test_split()

Creating working directory: cache-dir/Allstate
Please note: subsequent calls to `get_train_test_split` will read cached binary data, and thus be much faster.
Preprocessing Allstate dataset.


/Users/tpa/anaconda3/envs/snapenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Writing binary Allstate dataset (cache) to disk.


In [5]:
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

Number of examples: 9229003
Number of features: 87
Number of classes:  2


In [6]:
lr = LogisticRegression(fit_intercept=False, n_jobs=4)
t0 = time.time()
lr.fit(X_train, y_train)
t_fit_sklearn = time.time()-t0
score_sklearn = roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])
print("Training time (sklearn): %6.2f seconds" % (t_fit_sklearn))
print("ROC AUC score (sklearn): %.4f" % (score_sklearn))

Training time (sklearn): 39241.85 seconds
ROC AUC score (sklearn): 0.5872


In [7]:
lr = SnapLogisticRegression(fit_intercept=False, n_jobs=4)
t0 = time.time()
lr.fit(X_train, y_train)
t_fit_snapml = time.time()-t0
score_snapml = roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])
print("Training time (snapml): %6.2f seconds" % (t_fit_snapml))
print("ROC AUC score (snapml): %.4f" % (score_snapml))

Training time (snapml):   5.79 seconds
ROC AUC score (snapml): 0.5872


In [8]:
speed_up = t_fit_sklearn/t_fit_snapml
score_diff = (score_snapml-score_sklearn)/score_sklearn
print("Speed-up:                %.1f x" % (speed_up))
print("Relative diff. in score: %.4f" % (score_diff))

Speed-up:                6772.9 x
Relative diff. in score: -0.0000


## Disclaimer

Performance results always depend on the hardware and software environment. 

This notebook was run on the following machine:
* OS: MacOS 11.1 (Big Sur)
* CPU: 2.3 GHz Quad-Core Intel Core i7
* Memory: 32GB

The versions of the relevant software packages are given below:

In [9]:
import snapml
import sklearn
print("scikit-learn version: %s" % (sklearn.__version__))
print("      snapml version: %s" % (snapml.__version__))

scikit-learn version: 0.23.2
      snapml version: 1.7.0
